In [2]:
%run '01-data-collection.ipynb'

ModuleNotFoundError: No module named 'nbformat'

In [3]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline, FunctionTransformer
from sklearn.preprocessing import MinMaxScaler

In [14]:
# Definition of paths
data_dir = os.path.join(os.getcwd(), '../Dataset')

raw_data_dir = os.path.join(data_dir, 'raw')
ids_mapping = os.path.join(raw_data_dir, 'IDS_mapping.csv')
diabetes_data = os.path.join(raw_data_dir, 'diabetic_data.csv')

# Temporary-files created (based on the definition document) for the respective attribute/feature
working_dir = os.path.join(os.getcwd(), '../Temp')
mapping_data_dir = os.path.join(working_dir, 'Mapping')
os.makedirs(mapping_data_dir, exist_ok=True)

admission_type_mapping_data = os.path.join(mapping_data_dir, 'admission_type_mapping.csv')
discharge_disposition_data = os.path.join(mapping_data_dir, 'discharge_disposition_mapping.csv')
admission_source_mapping_data = os.path.join(mapping_data_dir, 'admission_source_mapping.csv')

In [5]:
# Lets load the main dataset
df = pd.read_csv(diabetes_data)

In the dataset, there are "?" sign that represented the NaN values. We shall replace that with np.nan.

In [10]:
# find out how many columns have '?' and the count of such values in different columns
na_summary = (df == '?').sum()
na_summary[na_summary != 0]

race                  2273
weight               98569
payer_code           40256
medical_specialty    49949
diag_1                  21
diag_2                 358
diag_3                1423
dtype: int64

In [12]:
# Replace all ?s with NaN
df.replace('?', pd.NA, inplace=True)

Lets look at the target feature: 'Readmitted'. The columnt represents if the patient was readmitted.


In [ ]:
# Are there any na's in the column?
df['readmitted'].isna().sum()

np.int64(0)

In [16]:
# What values are contained in the file
df['readmitted'].value_counts()

readmitted
NO     54864
>30    35545
<30    11357
Name: count, dtype: int64

If the patient was readmitted in less than 30 days, then the value is "<30"
If the patient was readmitted in more than 30 days, then the value is ">30"
If the patient was never readmitted, then the value is NO
Since we want to determine whether the patient was readmitted irrespective of number of days, hence

If the patient was never readmitted, then the value is 0.
If the patient was readmitted, then value is 1.

In [ ]:
# Pre-process Re-admitted column. If the value is NO, replace it with 0 otherwise replace it with 1. Treat this as a categorical variable.
df['readmitted'] = df['readmitted'].apply(lambda x: 0 if x == 'NO' else 1)